<a href="https://colab.research.google.com/github/focussed/etivity1/blob/master/Etivity2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Etivity 2

The goal of this lab is to prepare a dataset for a machine learning algorithm.   There are a number of procedures needed on the data set to complete this.

These include:


*   Performing a visual EDA on the dataset to get a visual feel for the data being treated.  Is there any glaring parts of the data missing or inconsistent (e.g. more than one piece of information per cell or large amounts of missing cells of data in a column).  This was done in excel and the dataset visually appears ok.  A diff shows it is an identical dataset to Etivity1.

## Filling the blank numeric data cells
I am taking a three step path to filling in the missing data.
1) Ensure the existing data is within an expected range.
This was done with the EDA in Excel and the df.describe() function.
2) Determine the missing data and analysis it for the next step.
This was done using df.isna().sum() and we can see that the missing fields are with :
- age (12)
- education (104)
- contact (191)
- poutcome(494)

3) With the missing data, we need to decide whether to delete the rows containing the missing data or to 'fill in' the missing data.  This will depend on each field.

Age field could be attempted to be filled out using an algorithm utilising mean values.
Education needs more consideration as there are 104 missing.  However, keeping the same % of primary, secondary and third level counts and filling in the 104 values may work.  Also deleting the rows may be ok as it is only 5% of the data.

## Features I intend to use
In order to fill missing numericial data, I expect to use a library like scikit-learn library or xgboost or a simple mean determination algorithm.  I will need to understand their pros and cons first.

I expect to test my final dataset for the ML stage by using a set of asserts as tests to ensure the final dataset is integral.

I expect to consider the use of pipelines to fill in missing data, once I understand their operation fully.

I expect to look at mapping text values to numerical values using the map feature.

I expect to identify and treat any outliers in the data as per the example in the lab2 notebook.


A describe() call is used to check the ranges of numerical fields.



In [0]:
# The Python modules uses in this tutorial are:

# - Pandas
# - Numpy
# - Matplotlib
# - Seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sort=True

url_train = 'https://raw.githubusercontent.com/focussed/etivity1/master/bank.csv'
df = pd.read_csv(url_train)

In [4]:
df.describe()

,age,balance,day,duration,campaign,pdays,previous
count,1988.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,41.753018,1413.663500,13.851500,292.020500,1.909500,167.896000,2.561500
std,12.724358,3131.224213,9.712189,221.557295,1.378862,131.754126,3.400735
min,18.000000,-980.000000,1.000000,7.000000,1.000000,-1.000000,0.000000
25%,32.000000,201.500000,5.000000,146.000000,1.000000,75.750000,1.000000
50%,38.000000,551.000000,12.000000,236.000000,1.000000,182.000000,2.000000
75%,50.000000,1644.500000,23.000000,379.000000,2.000000,251.000000,3.000000
max,93.000000,81204.000000,31.000000,1823.000000,11.000000,854.000000,55.000000


Ages appear to be in range as does balance the the other numeric fields.  pdays = -1 is one item that sticks out but there is an explanation for this.

In [5]:
df.isna().sum()

age            12
job            10
marital         0
education     104
default         0
balance         0
housing         0
loan            0
contact       191
day             0
month           0
duration        0
campaign        0
pdays           0
previous        0
poutcome      454
subscribed      0
dtype: int64